
# WA Conservation & Sensitive Lists

## List Identifiers
* Conservation list: __[dr2201](https://lists.ala.org.au/speciesListItem/list/dr2201)__
* Sensitive list: __[dr467](https://lists.ala.org.au/speciesListItem/list/dr467)__

## Documentation
* __[Home Page](https://www.dpaw.wa.gov.au/plants-and-animals/threatened-species-and-communities)__

## Downloads
* __[Fauna](https://www.dpaw.wa.gov.au/images/documents/plants-animals/threatened-species/Listings/Threatened%20and%20Priority%20Fauna%20List.xlsx)__
* __[Flora](https://www.dpaw.wa.gov.au/images/documents/plants-animals/threatened-species/Listings/Threatened%20and%20Priority%20Flora%20List%205%20December%202018.xlsx)__

**Note:**
* WA uses the same data files for both conservation and sensitive lists.
* Fauna and Flora are in separate files and have different formats.
* Files are processed separately and then concatenated into a single file.


Import libraries
Set project directories
Set Excel file locations

In [ ]:
import pandas as pd
import numpy as np

In [9]:
projectdir = "/Users/oco115/PycharmProjects/auth-lists-updates/WA-2022-10/"
basedir = "/Users/oco115/PycharmProjects/authoritative-lists/WA-2022-10/"
# faunafile = "https://www.dpaw.wa.gov.au/images/Threatened%20and%20Priority%20Fauna%20List%207%20October%202022.xlsx"
# florafile = "https://www.dpaw.wa.gov.au/images/Threatened%20and%20Priority%20Flora%20List%206%20October%202022.xlsx"

faunafile = projectdir + "source-data/Threatened and Priority Fauna List 7 October 2022-fixed.xlsx"
florafile = projectdir + "source-data/Threatened and Priority Flora List 6 October 2022-fixed.xlsx"



## Process Fauna

In [ ]:
print('Process Fauna')
print("Loading WA Fauna List")
fauna = pd.read_excel(faunafile,skiprows=0)
print("Finished Fauna download")
fauna

In [11]:
fauna.columns

Index(['Scientific name', 'Common name', 'Class', 'WA listing',
       'National listing', 'Kimberley', 'Pilbara', 'Goldfields', 'Midwest',
       'Wheatbelt', 'South Coast', 'Swan', 'South West', 'Warren',
       'WA listing.1', 'Notes'],
      dtype='object')

Map DWC terms

In [ ]:
fauna = fauna.rename(columns=
{
        'Scientific name': 'scientificName',
        'Common name': 'vernacularName',
        'Class': 'class',
        'WA listing': 'sourceStatus',
        'National listing': 'epbc status',
        'WA listing note': 'wa listing note',
        'Notes': 'taxonRemarks',
        'Kimberley': 'KIMB',
        'Pilbara': 'PILB',
        'Goldfields': 'GOLD',
        'Midwest': 'MWST',
        'Wheatbelt': 'WHTB',
        'South Coast': 'SCST',
        'Swan': 'SWAN',
        'South West': 'SWST',
        'Warren': 'WARR'
    })
fauna.columns

Build 'DBCA Region' field. Original data has a column for each locality with 'X' marked in cells where the species is found

In [ ]:
df = fauna[['KIMB', 'PILB', 'GOLD','MWST', 'WHTB', 'SCST', 'SWAN', 'SWST', 'WARR']].copy()
v = np.where(df == "X")
fauna['dbca region'] = (pd.DataFrame(df.columns[v[1]], index=df.index[v[0]], columns=['result']).groupby(level=0).agg(','.join).reindex(df.index))
# drop location columns
fauna = fauna.drop(['KIMB',
                    'PILB',
                    'GOLD',
                    'MWST',
                    'WHTB',
                    'SCST',
                    'SWAN',
                    'SWST',
                    'WARR'
                    ], axis=1)

Create status transforming sourceStatus codes

In [ ]:
print('Transforming fauna')
fauna['status'] = fauna['sourceStatus'].replace(['P1', 'P2', 'P3', 'P4', 'VU', 'MI', 'CR', 'EN',
                                                'OS', 'EX', 'EW', 'CD',
                                                'MI (& VU or CR at subsp. level)',
                                                'VU (& MI at sp. level)',
                                                'CR (& MI at sp. level)',
                                                'MI & P4'],
                                                ['P1 - Poorly known species', 'P2 - Poorly known species',
                                                'P3 - Poorly known species', 'P4 - Poorly known species',
                                                'Vulnerable', 'Migratory', 'Critically Endangered', 'Endangered',
                                                'Other specially protected fauna', 'Extinct', 'Extinct in Wild',
                                                'Special Conservation Interest', 'Migratory', 'Vulnerable',
                                                'Critically Endangered', 'Migratory & P4'
                                                ])

## Process Flora

In [ ]:
print('Processing Flora')
print("Loading WA Flora List")
flora = pd.read_excel(florafile,skiprows=0)
print("Finished Flora download")
flora

In [32]:
flora.columns


Index(['Name ID', 'Taxon', ' WA Status', 'WA Rank', 'WA IUCN Criteria', 'EPBC',
       'DBCA Region', 'DBCA District', 'Distribution', 'Flowering Period',
       'Recovery Plan', 'Notes'],
      dtype='object')

### Transform flora
**Data Transformations**
**STATUS**	Conservation status of taxon - refer to definitions.
- 	T -	Threatened Flora (Declared Rare Flora - Extant)
- 	X -	Presumed Extinct (Declared Rare Flora - Extinct)
-	1 -	Priority One - Poorly known Species
- 	2 -	Priority Two - Poorly known Species
- 	3 -	Priority Three - Poorly known Species
- 	4 -	Priority Four - Rare, Near Threatened and other species in need of monitoring

**RANK** The threat category the taxon is recognised as in Western Australia (see definitions)
-	CR - Critically Endangered
-	EN - Endangered- 	VU	Vulnerable
-	EX - Extinct

**EPBC** The category that the taxon is listed under the Commonwealth's Environmental Protection and Biodiversity Conservation Act 1999. Note this list is maintained by the Commonwealth and the official list should be sourced at the Commonwealth's website
- 	CR - Critically Endangered
- 	E  - Endangered
- 	V  - Vulnerable
- 	X  - Extinct


In [ ]:
print('Transforming flora')
flora = flora.drop(['Flowering Period',
                    'WA IUCN Criteria',
                    'Recovery Plan'
                    ], axis=1)

flora = flora.rename(columns=
                     {'Name ID': 'taxonId',
                      'Taxon': 'scientificName',
                      ' WA Status': 'sourceStatus',
                      'WA Rank': 'wa rank',
                      'DBCA Region': 'dbca region',
                      'DBCA District': 'dbca district',
                      'Distribution': 'verbatimLocality',
                      'EPBC': 'epbc status',
                      'Notes': 'taxonRemarks'
                      })
flora['status'] = flora['sourceStatus'].replace(['T', 'X', 1, 2, 3, 4],
                                                ['Threatened Flora', 'Presumed Extinct',
                                                 'P1 - Poorly known species', 'P2 - Poorly known species',
                                                 'P3 - Poorly known species', 'P4 - Poorly known species'])

In [34]:
flora.columns

Index(['taxonId', 'scientificName', 'sourceStatus', 'wa rank', 'epbc status',
       'dbca region', 'dbca district', 'verbatimLocality', 'taxonRemarks',
       'status'],
      dtype='object')

Concatenate dataframes

In [ ]:
dfsightings = pd.DataFrame()
dfsightings = pd.concat([fauna, dfsightings], axis=0, ignore_index=True, sort=False)
dfsightings['verbatimLocality'] = ""
dfsightings['sensitivityZoneId'] = "WA"
dfsightings = pd.concat([flora, dfsightings], axis=0, ignore_index=True, sort=False)

In [36]:
dfsightings.columns

Index(['taxonId', 'scientificName', 'sourceStatus', 'wa rank', 'epbc status',
       'dbca region', 'dbca district', 'verbatimLocality', 'taxonRemarks',
       'status', 'vernacularName', 'class', 'WA listing.1',
       'sensitivityZoneId'],
      dtype='object')

Write dataframe to both conservation and sensitive CSV files - UTF-8 encoding

In [ ]:
print('writing conservation and sensitive lists')
dfsightings.to_csv(projectdir + "current-lists/conservation-lists/WA-conservation-2022-10.csv", encoding="UTF-8",index=False)
dfsightings.to_csv(projectdir + "current-lists/sensitive-lists/WA-sensitive-2022-10.csv", encoding="UTF-8", index=False)
print('Completed writing')

In [ ]:
# Required for previous versions of files
# update the encoding on the final file / conservation is same as sensitive
# fileurl = "/Users/new330/IdeaProjects/authoritative-lists/current-lists/sensitive-lists/WA-sensitive.csv"
# wa = pd.read_csv(fileurl,encoding='cp1252')
# wa.to_csv(fileurl, encoding="UTF-8",index=False)
# wa.to_csv("/Users/new330/IdeaProjects/authoritative-lists/current-lists/conservation-lists/WA-conservation.csv", encoding="UTF-8",index=False)
#
# wa = wa.WATaxonId.astype(int)
# wa